## LabJack Example\
Demonstrates reading 4 analog inputs (AINs) in a loop from a LabJack.

Relevant Documentation:
 
LJM Library:
    LJM Library Installer:
        https://labjack.com/support/software/installers/ljm
    LJM Users Guide:
        https://labjack.com/support/software/api/ljm
    Opening and Closing:
        https://labjack.com/support/software/api/ljm/function-reference/opening-and-closing
    Multiple Value Functions(such as eWriteNames):
        https://labjack.com/support/software/api/ljm/function-reference/multiple-value-functions
    Timing Functions(such as StartInterval):
        https://labjack.com/support/software/api/ljm/function-reference/timing-functions
 
T-Series and I/O:
    Modbus Map:
        https://labjack.com/support/software/api/modbus/modbus-map
    Analog Inputs:
        https://labjack.com/support/datasheets/t-series/ain

In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt

from labjack import ljm

In [ ]:
# Open first found LabJack
handle = ljm.openS("T7", "ANY", "ANY")  # T7 device, Any connection, Any identifier

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Setup and call eWriteNames to configure AIN0 and AIN1 on the LabJack.
# LabJack T7 and other devices configuration

# AIN0, AIN1, AIN2, AIN3:
#   Negative channel = single ended (199)
#   Range: +/-10.0 V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0)
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_GAIN", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US",
         "AIN2_NEGATIVE_CH", "AIN2_RANGE", "AIN2_RESOLUTION_INDEX", "AIN2_SETTLING_US",
         "AIN3_NEGATIVE_CH", "AIN3_RANGE", "AIN3_RESOLUTION_INDEX", "AIN3_SETTLING_US"]
aValues = [199, 10.0, 10.0, 8, 10,
           199, 10.0, 8, 10,
           199, 10.0, 8, 10,
           199, 10.0, 8, 10]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

In [ ]:
print("\nSet configuration:")
for i in range(numFrames):
    print("    %s : %f" % (names[i], aValues[i]))


In [ ]:
# Setup and call eWriteName to write a value to the LabJack.
name = "DAC0"
value = 2.0  # 2.5 V
ljm.eWriteName(handle, name, value)

print("\neWriteName: ")
print("    Name - %s, value : %f" % (name, value))

In [ ]:
# Read AIN0(2.5V) and AIN1-3 (XYZ Accel)

loopAmount = 10
# Read AIN's from the LabJack with eReadNames in a loop.
names = ["AIN0", "AIN1", "AIN2", "AIN3"]
numFrames = 4
intervalHandle = 1
ljm.startInterval(intervalHandle, 50000)  # Delay between readings (in microseconds)
plotResults = []
start = time.time()
for n in range(loopAmount):
    results = ljm.eReadNames(handle, numFrames, names)
    plotResults.append(results)
    ljm.waitForNextInterval(intervalHandle)
    stamp = time.time()
    delay = stamp - start
    start = stamp

    print(delay, results)
#plotResults = (np.array(plotResults) - 2.50)

In [ ]:
#plt.plot(np.array(plotResults)[:,0], label=2.0V)
nStart = 0
nStop = 500
plt.plot(np.array(plotResults)[nStart:nStop,1], label = "X-accel")
plt.plot(np.array(plotResults)[nStart:nStop,2], label = "Y-accel")
plt.plot(np.array(plotResults)[nStart:nStop,3], label = "Z-accel")
plt.ylabel("Acceleration (g)")
plt.xlabel("T(mSec)")
plt.legend()
plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Test2_03Sep21.pdf")

In [ ]:
# Close handles
ljm.cleanInterval(intervalHandle)
ljm.close(handle)

In [ ]:
print(time.time())